<a href="https://colab.research.google.com/github/pavanvvce/100-Days-Of-ML-Code/blob/master/GPT-J-6B/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference with GPT-J-6B

In this notebook, we are going to perform inference (i.e. generate new text) with EleutherAI's [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/), which is a 6 billion parameter GPT model trained on [The Pile](https://arxiv.org/abs/2101.00027), a huge publicly available text dataset, also collected by EleutherAI. The model itself was trained on TPUv3s using JAX and Haiku (the latter being a neural net library on top of JAX).

[EleutherAI](https://www.eleuther.ai/) itself is a group of AI researchers doing awesome AI research (and making everything publicly available and free to use). They've also created [GPT-Neo](https://github.com/EleutherAI/gpt-neo), which are smaller GPT variants (with 125 million, 1.3 billion and 2.7 billion parameters respectively). Check out their models on the hub [here](https://huggingface.co/EleutherAI).

NOTE: this notebook requires at least 12.1GB of CPU memory. I'm personally using Colab Pro to run it (and set runtime to GPU - high RAM usage). Unfortunately, the free version of Colab only provides 10 GB of RAM, which isn't enough.

## Install dependencies

We will install Transformers from source for now.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 8.5 MB/s 
     |████████████████████████████████| 3.3 MB 64.6 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 895 kB 89.9 MB/s 


## Load model and tokenizer

First, we load the model from the hub. We select the "float16" revision, which means that all parameters are stored using 16 bits, rather than the default float32 ones (which require twice as much RAM memory). We also set `low_cpu_mem_usage` to `True` (which was introduced in [this PR](https://github.com/huggingface/transformers/pull/13466)), in order to only load the model once into CPU memory.

Next, we move the model to the GPU and load the corresponding tokenizer, which we'll use to prepare text for the model.

In [ ]:
import torch
from transformers import GPTJForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", low_cpu_mem_usage=True)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

## Inference

Here, we can provide a custom prompt, prepare that prompt using the tokenizer for the model (the only input required for the model are the `input_ids`). We then move the `input_ids` also to the GPU, and use the `.generate()` method to generate tokens autoregressively. Note that this method supports various decoding methods, including beam search and top k sampling. All details can be found in this [blog post](https://huggingface.co/blog/how-to-generate).

In [ ]:
prompt = "The Belgian national football team "
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The Belgian national football team  (,, ), known as the Blue and Whites, represents Belgium in international football competitions organised by FIFA and the governing body for football in Belgium, the Royal Belgian Football Association (,, ). It is coached by former Netherlands international and UEFA Euro 1984 winner Dick Advocaat, who was appointed in January 2018 after the departure of Michel Preud'homme. The Belgium team has been a force in international football since the 1960s, winning the 1974 FIFA World Cup and Euro 2000. It also qualified for UEFA Euro 2020. The Belgium national team is based and plays its games in the Antwerp region, with Rupelstad Stadion, home to its first- and second-tier matches, as a regular venue.

Belgium played its first official international match on 21 January 1920, losing 0–2 to the Netherlands in Rotterdam. Belgium and the Netherlands have played each other in 15 matches, with the Dutch winning 10 times and


Note that one of the most interesting properties of large GPT models is that they are capable of so-called "few-shot learning". This means that, given only a few examples in a text prompt, the model is capable of quickly generalizing to new, unseen examples. So you can use this model for example to do few-shot text classification, as follows:

In [ ]:
prompt = """
Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment:  neutral

#####

Sentence: This movie is really fun to watch.
Sentiment: positive

#####

Sentence: I would recommend this movie.
Sentiment:  neutral

#####

Sentence: This movie was good for a popcorn flick.
Sentiment: positive

#####

Sentence: I liked this movie.
Sentiment:  neutral

#####

Sentence: I hated this movie, it was terrible.
Sentiment:  negative

#####

Sentence: This


Another note: as GPT-J-6B was trained on The Pile (which includes a lot of Github code), the model is capable of performing code generation (similar to OpenAI's Codex model). Here's an example:

In [ ]:
prompt = """Instruction: Generate a Python function that lets you reverse a list.

Answer: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruction: Generate a Python function that lets you reverse a list.

Answer: 
import random

def reverse_list(aList):
        i = len(aList)
        x = 0
        while x < len(aList):
                if aList[x] < aList[0]:
                        aList[x] = random.choice(aList[x])
                else:
                        aList[x] = random.choice(aList[x])




## Batched inference

You can also perform inference on batches of prompts, as follows:

In [ ]:
# this is a single input batch with size 3
texts = ["Once there was a man ", "The weather today will be ", "The best football player in the world is "]

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
with torch.no_grad():
    generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_length=50)
generated_texts = tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True)

for text in generated_texts:
  print("---------")
  print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------
Once there was a man  
who had many wonderful talents, and one of them was that he was tall and straight, and so he walked over hill and dale, and wherever he walked he could make the grass grow wherever
---------
The weather today will be   cool with a chance of snow in the overnight hours.  
Mostly cloudy skies will be on tap for the day. Overnight lows will dip to 
around freezing in the lower elev
---------
The best football player in the world is ____:

A) Lionel Messi

B) Cristiano Ronaldo

C) Zlatan Ibrahimovic

Now let's try making sense of the questions. You should know that "


In [7]:
import bs4 as bs
import urllib.request
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Machine_learning')
ml_wiki = scrapped_data.read()
ml_wiki_parsed = bs.BeautifulSoup(ml_wiki,'lxml')
paragraphs = ml_wiki_parsed.find_all('p')
article_text = ""
for p in paragraphs:
    article_text += p.text


In [5]:
# Cleaing the text
ml_wiki_cleaned = article_text.lower()
ml_wiki_cleaned = re.sub('[^a-zA-Z]', ' ', ml_wiki_cleaned)
ml_wiki_cleaned = re.sub(r'\s+', ' ', ml_wiki_cleaned)


In [8]:
# Preparing the dataset
all_sentences = nltk.sent_tokenize(ml_wiki_cleaned)
all_words = [nltk.word_tokenize(sent) for sent in all_sentences]
# Removing Stop Words
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]


In [9]:
from gensim.models import Word2Vec
word2vec = Word2Vec(all_words, min_count=2)


In [10]:
vocabulary = word2vec.wv.vocab
print(vocabulary)
# you can use the print command to observe the output


{'machine': <gensim.models.keyedvectors.Vocab object at 0x7ffb4b01b390>, 'learning': <gensim.models.keyedvectors.Vocab object at 0x7ffb4b01b610>, 'ml': <gensim.models.keyedvectors.Vocab object at 0x7ffb4b01b4d0>, 'study': <gensim.models.keyedvectors.Vocab object at 0x7ffb4b01b910>, 'computer': <gensim.models.keyedvectors.Vocab object at 0x7ffb48a69350>, 'algorithms': <gensim.models.keyedvectors.Vocab object at 0x7ffb487f53d0>, 'improve': <gensim.models.keyedvectors.Vocab object at 0x7ffb487fab50>, 'experience': <gensim.models.keyedvectors.Vocab object at 0x7ffb48795c50>, 'use': <gensim.models.keyedvectors.Vocab object at 0x7ffb493cbc10>, 'data': <gensim.models.keyedvectors.Vocab object at 0x7ffb487f5390>, 'seen': <gensim.models.keyedvectors.Vocab object at 0x7ffb48795c10>, 'part': <gensim.models.keyedvectors.Vocab object at 0x7ffb48795cd0>, 'artificial': <gensim.models.keyedvectors.Vocab object at 0x7ffb48795d10>, 'intelligence': <gensim.models.keyedvectors.Vocab object at 0x7ffb48795d

In [11]:
v1 = word2vec.wv['artificial']

In [12]:
print("length of the vector representation is:" , len(v1))
print(v1)

length of the vector representation is: 100
[ 3.9240872e-04 -1.8157914e-03  9.0120191e-04  3.0787168e-03
 -4.6103485e-03 -2.5700609e-04  5.0966023e-03 -3.2628269e-03
  7.1055172e-03 -1.6627562e-03  5.5535794e-03  2.9866556e-03
  4.7026975e-03  3.3454548e-03  1.8831971e-03 -1.9817364e-03
  3.0388376e-03 -2.2525273e-03 -6.8433946e-03 -6.0417955e-03
  2.2862328e-03 -6.2465476e-04  4.0401570e-03  3.9240769e-03
 -6.7369542e-03  5.2420730e-03 -3.6788080e-03 -6.8658004e-03
 -7.7536847e-03  5.3822005e-04  5.9461067e-03 -1.4341138e-03
 -2.5525340e-03 -6.2959371e-03  1.4044734e-03  8.7960474e-03
 -1.3605703e-04 -1.7069440e-03 -8.5942243e-04  6.7372178e-03
  4.5978264e-03  4.5227292e-03  2.4724582e-03  3.1507802e-03
 -7.2799932e-04 -4.0240875e-06 -3.2827447e-03  3.6934461e-03
 -3.4427627e-03 -1.7978899e-03 -5.4114871e-03  1.3384885e-03
  2.8414687e-03  6.0986192e-04  1.6595819e-03 -3.9330381e-03
 -3.9278218e-03  5.4518930e-03  5.3035334e-04  1.2200925e-03
  3.0579660e-03  2.1389632e-03 -1.0356585